In [20]:
from tools.bio import *

In [21]:
!java -jar /cluster/home/ifiddes/jvarkit/dist-1.133/biostar94573.jar  -R Notch2NL_consensus n2nl-100kb-with-consensus.fa  > consensus.raw.vcf

[INFO/Biostar94573] 2017-11-11 20:11:22 "Starting JOB at Sat Nov 11 20:11:22 PST 2017 com.github.lindenb.jvarkit.tools.biostar.Biostar94573 version=d31e94729837485c0eeaa8bf00300fee7dd25b88  built=2015-08-15:12-08-53"
[INFO/Biostar94573] 2017-11-11 20:11:22 "Command Line args : -R Notch2NL_consensus n2nl-100kb-with-consensus.fa"
[INFO/Biostar94573] 2017-11-11 20:11:22 "Executing as ifiddes@kolossus on Linux 2.6.32-642.11.1.el6.x86_64 amd64; Java HotSpot(TM) 64-Bit Server VM 1.8.0_45-b14"
[INFO/Biostar94573] 2017-11-11 20:11:22 "Reading from n2nl-100kb-with-consensus.fa"
[INFO/Biostar94573] 2017-11-11 20:11:22 "format : Fasta"
[INFO/Biostar94573] 2017-11-11 20:11:22 "Done"
[INFO/Biostar94573] 2017-11-11 20:11:22 "End JOB status=0 [Sat Nov 11 20:11:22 PST 2017] com.github.lindenb.jvarkit.tools.biostar.Biostar94573 done. Elapsed time: 0.01 minutes."


In [22]:
import vcf
vcf_path = 'consensus.raw.vcf'
vcf_recs = list(vcf.Reader(open(vcf_path)))

In [23]:
vcf_recs = [x for x in vcf_recs if x.is_snp]

In [25]:
# construct a list of markers with AB merged
r = []
from tools.fileOps import *
import pandas as pd
order = ['NOTCH2NLA', 'NOTCH2NLB', 'NOTCH2NLC', 'NOTCH2NLR', 'NOTCH2']
outf = open('snps_only.vcf', 'w')
writer = vcf.Writer(outf, vcf.Reader(open(vcf_path)))
for v in vcf_recs:
    if v.is_indel:
        continue
    vals = {x.sample : int(x.gt_alleles[0]) for x in v.samples}
    vals = [v.POS - 1] + [vals[x] for x in order]
    r.append(vals)
    writer.write_record(v)
outf.close()
df = pd.DataFrame(r, columns=['position', 'A', 'B', 'C', 'D', 'N'])
df['AB'] = df['A'] + df['B']
df = df.drop(['A', 'B'], axis=1)
df[['position', 'AB', 'C', 'D', 'N']].set_index('position').to_csv('hg38_features_ab_merge.txt', sep='\t')

In [26]:
# construct a list of markers with AB split
r = []
from tools.fileOps import *
import pandas as pd
order = ['NOTCH2NLA', 'NOTCH2NLB', 'NOTCH2NLC', 'NOTCH2NLR', 'NOTCH2']
for v in vcf_recs:
    if v.is_indel:
        continue
    vals = {x.sample : int(x.gt_alleles[0]) for x in v.samples}
    vals = [v.POS - 1] + [vals[x] for x in order]
    r.append(vals)
df = pd.DataFrame(r, columns=['position', 'A', 'B', 'C', 'D', 'N'])
df[['position', 'A', 'B', 'C', 'D', 'N']].set_index('position').to_csv('hg38_features.txt', sep='\t')

In [53]:
# merge feature sets
new_df = pd.read_csv('NA12878_NA19240_NA24385_H9_AB.100kb.features.tsv', sep='\t')

In [56]:
df['A'] = [1 if x > 0 else 0 for x in df['A']]
df['B'] = [1 if x > 0 else 0 for x in df['B']]
df['C'] = [1 if x > 0 else 0 for x in df['C']]
df['D'] = [1 if x > 0 else 0 for x in df['D']]
df['N'] = [1 if x > 0 else 0 for x in df['N']]

merged = pd.merge(new_df, df, on='position', how='outer').fillna(0).sort_values('position')
merged['position'] = map(int, merged['position'])

In [57]:
merged.set_index('position').to_csv('merged_features.tsv', sep='\t')